<h1 align="center"><img align="center" src="https://geoparse.io/graphics/geoparse_logo.png" alt="GeoParse Logo" width="200"/></h1>
<h1 align="center">GeoParse</h1>
<h3 align="center">It's all about points <img src="https://geoparse.io/graphics/point.png" width="10"/> lines <img src="https://geoparse.io/graphics/line.png" width="40"/> and polygons <img src="https://geoparse.io/graphics/polygon.png" width="30"/></h3>


#### [Website](http://geoparse.io) 
***

# Visualization

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/geoparse/geoparse/main?labpath=tutorials%2F00_visualization.ipynb)

This notebook demonstrates how to display:

* Points, lines, and polygons.
* Their heatmaps and clusters.
* Their coverage areas using geospatial cells such as H3, S2, and Geohash.
* Geospatial cells on a map based on their indexes.
* OSM roads and buildings using their IDs.
***

In [ ]:
import os
import sys

sys.path.append(os.path.abspath("../geoparse/"))
import warnings

import geopandas as gpd
import pandas as pd

from geoparse import plp

warnings.filterwarnings("ignore")

## Read from Various File Formats

GeoParse creates interactive maps with multiple tile layers and supports data visualization from `CSV` and GIS formats like `Shapefile`, `GPKG`, `GeoJSON`, and `GeoParquet`. 

The `plp` (point, line, polygon) function in the `Karta` class (Swedish for "map") enables geometry visualization using either a pandas `DataFrame` or a GeoPandas `GeoDataFrame`. When given a `DataFrame`, `plp` automatically detects columns containing **lat** and **lon** (case-insensitive) to use as latitude and longitude for plotting points on the map.

You can switch between different tile layers using the options in the top right corner of the map.

Available Tile Layers:
* **Light:** A clean, minimalistic basemap that highlights overlaid data.
* **Dark:** A high-contrast, dark-themed map ideal for vibrant overlays and nighttime visualization.
* **Outdoors:** Designed for outdoor activities, featuring trails, elevation contours, and natural landmarks.
* **Satellite:** Displays high-resolution satellite imagery for real-world context and detailed analysis.
* **OSM:** A versatile map powered by OpenStreetMap, displaying roads, buildings, and points of interest.

In [ ]:
%%time
df = pd.read_csv("https://geoparse.io/data/great_britain_road_casualties-2023.csv")
df.head()

In [ ]:
plp(df)

In [ ]:
plp(df, point_color="speed_limit")

In [ ]:
plp(df, point_color="speed_limit", point_popup={"Speed limit": "speed_limit"})

In [ ]:
plp(df, heatmap=True)

In [ ]:
plp(df, cluster=True)

In [ ]:
plp(df, heatmap=True, cluster=True)

If no columns contain `lat` and `lon` keywords, or if more than two columns contain these keywords, you must explicitly specify the latitude and longitude using the `y` and `x` parameters, respectively, e.g., `plp(df, x="easting", y="northing")`. Note that `plp` assumes all data is in the [EPSG:4326](https://epsg.io/4326) projection. 

For a `GeoDataFrame`, the `plp` function can render Shapely objects such as `Point`, `LineString`, `Polygon`, and `MultiPolygon`.

## Read a Polygon from Geo File

In [ ]:
%%time
gdf = gpd.read_file("https://geoparse.io/data/london.geojson")
gdf

In [ ]:
plp(gdf)

In [ ]:
plp(gdf, centroid=True)

In [ ]:
plp(gdf, h3_res=6)

In [ ]:
plp(gdf, h3_res=10, compact=True)

In [ ]:
plp(gdf, s2_res=12)

In [ ]:
plp(gdf, s2_res=14, compact=True)

In [ ]:
plp(gdf, geohash_res=5)

In [ ]:
plp(gdf, geohash_res=7, compact=True)

# Mutiple data frames

In [ ]:
plp([df, gdf], h3_res=6)

# Geospatial Cells

In [ ]:
import h3

lat, lon = 41.87, -87.78

# Get the H3 index at resolution 6 for the central point
h3_index = h3.latlng_to_cell(lat, lon, 6)

# Get adjacent H3 cells including the central cell itself
h3_cells = h3.grid_disk(h3_index, 1)  # k_ring with radius 1 returns the central cell + neighbors
h3_cells

In [ ]:
plp(cells=list(h3_cells), cell_type="h3")

# OSM Ways

In [ ]:
plp(osm_ways=[335265936, 53820456, 1117218957], s2_res=20, compact=True)

In [ ]:
plp(osm_ways=[260909736, 54454413])